# Quick draw classifier ✍️📖

The [Quick Draw Dataset](https://github.com/googlecreativelab/quickdraw-dataset) is a collection of 50 million drawings across 345 categories, contributed by players of the game Quick, Draw! - ***Google***.

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
# !gdown https://console.cloud.google.com/storage/browser/quickdraw_dataset/full/simplified

In [ ]:
classes = ["banana", "apple", "computer", "crown", "clock", "circle", "pants", "rainbow", "sun", "snowman", "telephone", "umbrella", "tree", "bridge", "cloud"]

In [ ]:
len(classes)

15

In [ ]:
os.mkdir("data")

In [ ]:
import urllib.request
base = 'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/'
for c in classes:
  cls_url = c.replace('_', '%20')
  path = base+cls_url+'.npy'
  print(path)
  urllib.request.urlretrieve(path, "data/" + c + '.npy')

https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/banana.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/apple.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/computer.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/crown.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/clock.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/circle.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/pants.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/rainbow.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/sun.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/snowman.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/telephone.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/umbrella.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bi

In [ ]:
import glob

def load_data(root, vfold_ratio=0.2, max_items_per_class=20000):
    all_files = glob.glob(os.path.join(root, '*.npy'))

    #initialize variables 
    x = np.empty([0, 784])
    y = np.empty([0])
    class_names = []
    
    #load a subset of the data to memory 
    for idx, file in enumerate(all_files):
        data = np.load(file)
        data = data[0: max_items_per_class, :]
        labels = np.full(data.shape[0], idx)

        x = np.concatenate((x, data), axis=0)
        y = np.append(y, labels)

        class_name, ext = os.path.splitext(os.path.basename(file))
        class_names.append(class_name)

    data = None
    labels = None

    #separate into training and testing 
    permutation = np.random.permutation(y.shape[0])
    
    x = x[permutation, :]
    y = y[permutation]

    vfold_size = int(x.shape[0]/100*(vfold_ratio*100))
    print(vfold_size)
    X_test = x[0:vfold_size, :]
    y_test = y[0:vfold_size]

    X_train = x[vfold_size:x.shape[0], :]
    y_train = y[vfold_size:y.shape[0]]
    return X_train, y_train, X_test, y_test, class_names

In [ ]:
X_train, y_train, X_test, y_test, classes = load_data("data")

60000


In [ ]:
classes

['snowman',
 'apple',
 'telephone',
 'tree',
 'circle',
 'banana',
 'crown',
 'pants',
 'sun',
 'clock',
 'rainbow',
 'umbrella',
 'cloud',
 'bridge',
 'computer']

In [ ]:
X_train.shape

(240000, 784)

In [ ]:
import cv2

def preprocess(X_train, X_test):
  X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype("uint8")
  X_train = np.repeat(X_train, 3, -1)
  X_test = X_test.reshape(X_test.shape[0], 28, 28, 1).astype("uint8")
  X_test = np.repeat(X_test, 3, -1)

  X_train = cv2.bitwise_not(X_train)
  X_test = cv2.bitwise_not(X_test)

  X_train = X_train / 255
  X_test = X_test / 255

  return X_train, X_test

In [ ]:
X_train, X_test = preprocess(X_train, X_test)

In [ ]:
X_test[0].dtype

dtype('uint8')

In [ ]:
from google.colab.patches import cv2_imshow
cv2_imshow(X_test[3] * 255)

In [ ]:
X_test[0][17]

array([[1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        ],
       [0.2745098 , 0.2745098 , 0.2745098 ],
       [0.23921569, 0.23921569, 0.23921569],
       [1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        ],
       [1.

In [ ]:
X_test[0][17]

array([[255, 255, 255],
       [255, 255, 255],
       [255, 255, 255],
       [181, 181, 181],
       [  0,   0,   0],
       [223, 223, 223],
       [ 61,  61,  61],
       [ 27,  27,  27],
       [  0,   0,   0],
       [144, 144, 144],
       [155, 155, 155],
       [  0,   0,   0],
       [226, 226, 226],
       [255, 255, 255],
       [255, 255, 255],
       [255, 255, 255],
       [255, 255, 255],
       [138, 138, 138],
       [ 12,  12,  12],
       [255, 255, 255],
       [244, 244, 244],
       [161, 161, 161],
       [ 67,  67,  67],
       [  2,   2,   2],
       [123, 123, 123],
       [255, 255, 255],
       [255, 255, 255],
       [255, 255, 255]], dtype=uint8)

In [ ]:
import cv2
normalized = cv2.normalize(X_test[0], None, 0, 1, cv2.NORM_MINMAX, dtype=cv2.CV_8U)

In [ ]:
X_test[0]

In [ ]:
normalized = normalized * 255
normalized

array([[[255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.],
        ...,
        [255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.]],

       [[255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.],
        ...,
        [255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.]],

       [[255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.],
        ...,
        [255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.]],

       ...,

       [[255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.],
        ...,
        [255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.]],

       [[255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.],
        ...,
        [255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.]],

       [[255., 255., 255.],
        [255., 255., 255.],
        [255., 2

In [ ]:
from google.colab.patches import cv2_imshow
cv2_imshow(normalized)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization

In [ ]:
input_shape = (28, 28, 3)

# Build model
model_1 = tf.keras.models.Sequential([
  Input(shape=input_shape),
  Conv2D(32, (3, 3), activation="relu", padding="same"),
  BatchNormalization(),
  Conv2D(64, (3, 3), activation="relu", padding="same"),
  BatchNormalization(),
  MaxPool2D(2),
  Dropout(0.2),
  Conv2D(128, (3, 3), activation="relu", padding="same"),
  BatchNormalization(),
  MaxPool2D(2),
  Dropout(0.2),
  Conv2D(256, (3, 3), activation="relu", padding="same"),
  BatchNormalization(),
  MaxPool2D(2),
  Dropout(0.2),
  Dense(256, activation="relu"),
  BatchNormalization(),
  tf.keras.layers.Flatten(),
  Dense(15, activation="softmax")
])

# Compile model
model_1.compile(loss="sparse_categorical_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
model_1.fit(X_train, y_train, epochs=2, validation_data=(X_test, y_test))

Epoch 1/2
7500/7500 [==============================] - 64s 7ms/step - loss: 0.3057 - accuracy: 0.9149 - val_loss: 0.2107 - val_accuracy: 0.9408
Epoch 2/2
7500/7500 [==============================] - 54s 7ms/step - loss: 0.1905 - accuracy: 0.9457 - val_loss: 0.1717 - val_accuracy: 0.9518


In [ ]:
result = model_1.predict(tf.expand_dims(X_test[2], 0))
result.argmax()

12

In [ ]:
model_1.save("draw_model.h5")

In [ ]:
model = tf.keras.models.load_model("draw_model.h5")

In [ ]:
preds = model_1.predict(tf.expand_dims(X_test[2], axis=0))
idx = np.argmax(preds)
classes[idx]

'rainbow'

In [ ]:
classes[int(y_test[2])]

'umbrella'

In [ ]:
classes

['sun',
 'clock',
 'cloud',
 'telephone',
 'bridge',
 'tree',
 'computer',
 'pants',
 'snowman',
 'crown',
 'rainbow',
 'umbrella',
 'circle',
 'apple',
 'banana']